In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR1552488.assembly.len15.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/DECockroach.len15.pkl"

In [3]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [4]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,SSRHLQWDSTLSPHFCF,17
1,1,RGRCIRSSCPSSHLGWR,17
2,2,FCCYIYCMPVQNSIYAS,17
3,3,YSKFSLCQYYFKFCPYST,18
4,4,AADYKPEGLGFDSPWCHWNFSIGNPSGRTM,30
...,...,...,...
894303,894303,WPCTHNGDPCTHFGFSA,17
894304,894304,ETCGNQCNCDCADAGIN,17
894305,894305,LQPLNVLWHCMWKSLCEISECLHNLFPC,28
894306,894306,SCHRSCCKWHPSAVRHM,17


In [5]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

,ID,Sequence,length,reps
0,0,SSRHLQWDSTLSPHFCF,17,"[0.03517688065767288, -0.05165969580411911, -0..."
1,1,RGRCIRSSCPSSHLGWR,17,"[0.03560592234134674, -0.08346492797136307, -0..."
2,2,FCCYIYCMPVQNSIYAS,17,"[0.037034764885902405, 0.0007549189031124115, ..."
3,3,YSKFSLCQYYFKFCPYST,18,"[0.03289207071065903, -0.011171547695994377, -..."
4,4,AADYKPEGLGFDSPWCHWNFSIGNPSGRTM,30,"[0.02026762254536152, -0.06562212109565735, -0..."
...,...,...,...,...
894303,894303,WPCTHNGDPCTHFGFSA,17,"[0.037583060562610626, -0.11564307659864426, -..."
894304,894304,ETCGNQCNCDCADAGIN,17,"[0.03682224452495575, -0.07952786237001419, -0..."
894305,894305,LQPLNVLWHCMWKSLCEISECLHNLFPC,28,"[0.022418277338147163, -0.021197671070694923, ..."
894306,894306,SCHRSCCKWHPSAVRHM,17,"[0.03473144769668579, -0.043851062655448914, -..."


Remove Duplicate

In [8]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length,reps
0,0,SSRHLQWDSTLSPHFCF,17,"[0.03517688065767288, -0.05165969580411911, -0..."
1,1,RGRCIRSSCPSSHLGWR,17,"[0.03560592234134674, -0.08346492797136307, -0..."
2,2,FCCYIYCMPVQNSIYAS,17,"[0.037034764885902405, 0.0007549189031124115, ..."
3,3,YSKFSLCQYYFKFCPYST,18,"[0.03289207071065903, -0.011171547695994377, -..."
4,4,AADYKPEGLGFDSPWCHWNFSIGNPSGRTM,30,"[0.02026762254536152, -0.06562212109565735, -0..."
...,...,...,...,...
894303,894303,WPCTHNGDPCTHFGFSA,17,"[0.037583060562610626, -0.11564307659864426, -..."
894304,894304,ETCGNQCNCDCADAGIN,17,"[0.03682224452495575, -0.07952786237001419, -0..."
894305,894305,LQPLNVLWHCMWKSLCEISECLHNLFPC,28,"[0.022418277338147163, -0.021197671070694923, ..."
894306,894306,SCHRSCCKWHPSAVRHM,17,"[0.03473144769668579, -0.043851062655448914, -..."


In [7]:
df.to_pickle(plk)
tmp_df = pd.read_pickle(plk)
tmp_df

,ID,Sequence,length,reps
0,0,SSRHLQWDSTLSPHFCF,17,"[0.03517688065767288, -0.05165969580411911, -0..."
1,1,RGRCIRSSCPSSHLGWR,17,"[0.03560592234134674, -0.08346492797136307, -0..."
2,2,FCCYIYCMPVQNSIYAS,17,"[0.037034764885902405, 0.0007549189031124115, ..."
3,3,YSKFSLCQYYFKFCPYST,18,"[0.03289207071065903, -0.011171547695994377, -..."
4,4,AADYKPEGLGFDSPWCHWNFSIGNPSGRTM,30,"[0.02026762254536152, -0.06562212109565735, -0..."
...,...,...,...,...
894303,894303,WPCTHNGDPCTHFGFSA,17,"[0.037583060562610626, -0.11564307659864426, -..."
894304,894304,ETCGNQCNCDCADAGIN,17,"[0.03682224452495575, -0.07952786237001419, -0..."
894305,894305,LQPLNVLWHCMWKSLCEISECLHNLFPC,28,"[0.022418277338147163, -0.021197671070694923, ..."
894306,894306,SCHRSCCKWHPSAVRHM,17,"[0.03473144769668579, -0.043851062655448914, -..."
